In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# 데이터 로드
df = pd.read_csv('data/raw/train_taxi_tims_U.csv',encoding='cp949')

/var/folders/tg/ntvm9wc94h937xqtzftck9xc0000gn/T/ipykernel_2638/2216033192.py:8: DtypeWarning: Columns (1,5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/raw/train_taxi_tims_U.csv',encoding='cp949')


In [2]:
df.head()

,차량이름,승차시간,승차요일,승차X좌표,승차Y좌표,하차시간,하차X좌표,하차Y좌표,승차거리(m),할증여부,요금
0,xqbxch,NaN,Tuesday,127.367251,36.349866,20230620110821,127.367251,36.349866,0.00,미할증,3300
1,xqbxch,NaN,Tuesday,127.367263,36.349684,20230620111100,127.367263,36.349684,0.00,미할증,3300
2,xqbxch,NaN,Tuesday,0.000000,0.000000,20230620221810,127.343552,36.294073,8743.06,미할증,10300
3,xqbxch,NaN,Tuesday,127.340568,36.298811,20230620222720,127.350915,36.305854,1518.13,미할증,3300
4,xqbxch,NaN,Tuesday,127.351858,36.299839,20230620224715,127.374955,36.345087,7045.65,미할증,8000


In [3]:
# 사용 전 인스턴스 개수 출력
print("Before dropping NaN instances:", len(df))

# NaN 값을 가진 인스턴스 삭제
df.dropna(inplace=True)

# 사용 후 인스턴스 개수 출력
print("After dropping NaN instances:", len(df))

Before dropping NaN instances: 1156344
After dropping NaN instances: 566520


In [4]:
df.head()

,차량이름,승차시간,승차요일,승차X좌표,승차Y좌표,하차시간,하차X좌표,하차Y좌표,승차거리(m),할증여부,요금
393216,fcivmp,1970-01-01 05:37:11,Thursday,127.431169,36.349082,19700101053711,127.432874,36.330697,2618.33,미할증,5800
393217,mceayy,1970-01-01 05:37:11,Thursday,127.319037,36.363290,19700101053711,127.301159,36.337313,5102.27,미할증,8200
393218,vspxqy,1970-01-01 05:37:11,Thursday,127.314859,36.391744,19700101053711,127.350051,36.302253,12705.35,미할증,14400
393219,xqbsmx,1970-01-01 05:37:11,Thursday,127.369546,36.349814,19700101053711,127.382217,36.336760,2396.42,미할증,5200
393220,dmdlpm,1970-01-01 05:37:11,Thursday,127.377504,36.354857,19700101053711,127.356460,36.355071,2822.74,미할증,5600


In [5]:
# '승차시간'을 datetime 형식으로 변환
df['승차시간'] = pd.to_datetime(df['승차시간'])

# 년도가 2023 또는 2024가 아닌 인스턴스 삭제
df = df[df['승차시간'].dt.year.isin([2023, 2024])]

# 삭제 후 인스턴스 개수 출력
print("After deleting instances:", len(df))

After deleting instances: 558743


In [6]:
df.head()

,차량이름,승차시간,승차요일,승차X좌표,승차Y좌표,하차시간,하차X좌표,하차Y좌표,승차거리(m),할증여부,요금
400993,gouhxp,2023-03-31 23:41:17,Friday,127.314748,36.391978,2023-04-01 00:04:32,127.249854,36.508528,15313.0,2,18100
400994,hzdykr,2023-03-31 23:49:29,Friday,127.432939,36.331904,2023-04-01 00:06:57,127.333850,36.336123,11321.0,1,12260
400995,bfllzp,2023-04-01 00:04:45,Saturday,127.336672,36.307763,2023-04-01 00:11:28,127.345800,36.323435,2531.0,1,5040
400996,clspqc,2023-04-01 00:06:20,Saturday,127.411853,36.336721,2023-04-01 00:12:18,127.420696,36.346687,2086.0,1,4680
400997,dmpnpp,2023-04-01 00:05:10,Saturday,127.434345,36.328179,2023-04-01 00:13:34,127.461915,36.330595,3682.0,1,6120


In [7]:
df['할증여부'] = df['할증여부'].replace('미할증', 0)

In [8]:
# '승차시간'과 '하차시간'을 datetime으로 변환 (형식 지정, 잘못된 값은 NaT로 변환)
df['승차시간'] = pd.to_datetime(df['승차시간'], errors='coerce')
df['하차시간'] = pd.to_datetime(df['하차시간'], errors='coerce')

# '주행시간' 계산 및 새로운 열에 저장
df['주행시간'] = df['하차시간'] - df['승차시간']

In [9]:
# '주행시간'을 초 단위의 실수로 변환하여 '주행시간_초' 열에 저장
df['주행시간_초'] = df['주행시간'].dt.total_seconds()

In [10]:
df.head()

,차량이름,승차시간,승차요일,승차X좌표,승차Y좌표,하차시간,하차X좌표,하차Y좌표,승차거리(m),할증여부,요금,주행시간,주행시간_초
400993,gouhxp,2023-03-31 23:41:17,Friday,127.314748,36.391978,2023-04-01 00:04:32,127.249854,36.508528,15313.0,2,18100,0 days 00:23:15,1395.0
400994,hzdykr,2023-03-31 23:49:29,Friday,127.432939,36.331904,2023-04-01 00:06:57,127.333850,36.336123,11321.0,1,12260,0 days 00:17:28,1048.0
400995,bfllzp,2023-04-01 00:04:45,Saturday,127.336672,36.307763,2023-04-01 00:11:28,127.345800,36.323435,2531.0,1,5040,0 days 00:06:43,403.0
400996,clspqc,2023-04-01 00:06:20,Saturday,127.411853,36.336721,2023-04-01 00:12:18,127.420696,36.346687,2086.0,1,4680,0 days 00:05:58,358.0
400997,dmpnpp,2023-04-01 00:05:10,Saturday,127.434345,36.328179,2023-04-01 00:13:34,127.461915,36.330595,3682.0,1,6120,0 days 00:08:24,504.0


In [11]:
# 사용 전 인스턴스 개수 출력
print("Before dropping NaN instances:", len(df))

# NaN 값을 가진 인스턴스 삭제
df.dropna(inplace=True)

# 사용 후 인스턴스 개수 출력
print("After dropping NaN instances:", len(df))

Before dropping NaN instances: 558743
After dropping NaN instances: 494790


In [12]:
df.to_csv('data/processed/train_1.csv', index=False)